In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importation
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Lecture des données
data2015=pd.read_csv('../input/sea-building-energy-benchmarking/2015-building-energy-benchmarking.csv')
data2016=pd.read_csv('../input/sea-building-energy-benchmarking/2016-building-energy-benchmarking.csv')

In [ ]:
#Affichage des premières lignes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data2015.head()

In [ ]:
data2015.shape

In [ ]:
data2016.shape

Les deux jeux de données n'ont pas le même nombre de colonnes et de lignes, il y a un nettoyage à faire pour les adapter l'un à l'autre.

# Nettoyage

In [ ]:
#Noms de colonnes communes aux deux df
idColumns = data2015.columns.intersection(data2016.columns)

In [ ]:
#Noms de colonnes de data2015 non communs
set(data2015)-set(idColumns)

On constate des erreurs de formatage mais aussi des variables qui n'ont pas de correspondance. 

City Council Districts,  Seattle Police Department Micro Community Policing Plan Areas, 2010 Census Tracts, SPD Beats (2015) : données géopgraphique

Comment/Comments (2015) : incohérence du format du nom de variable

Location (2015) : Variable comprenant plusieurs indications sur la localisation que l'on retrouve dans les variables 'Longitude', 'Latitude', 'State', 'City', 'Adress', du fichier 2016

ZipCode/Zip Codes : incohérence du format du nom de variable

GHGEmissions Intensity et Total : incohérence du format du nom de variable

Other Fuel uses (2015) : probablement liés aux hydrocarbures liquides et solides, inexistants dans le fichier 2016 (Ne concerne que 17 lignes)


In [ ]:
# Formatage des noms de colonnes
data2015 = data2015.rename(columns = {'Comment':'Comments','GHGEmissions(MetricTonsCO2e)':'TotalGHGEmissions','GHGEmissionsIntensity(kgCO2e/ft2)':'GHGEmissionsIntensity','Zip Codes':'ZipCode'})

In [ ]:
#Retrait des caractères spéciaux des noms de colonnes
data2015.columns = data2015.columns.str.replace('[#,@,&,(,),/]', '')
data2016.columns = data2016.columns.str.replace('[#,@,&,(,),/]', '')

### Recherche de clé primaire

In [ ]:
#Test d'une clé candidate pour data2015
len(data2015['OSEBuildingID'].unique())-data2015.shape[0]

In [ ]:
#Test d'une clé candidate pour data2016
len(data2016['OSEBuildingID'].unique())-data2016.shape[0]

In [ ]:
# Clés communes aux deux fichiers
buildingID = set(data2015['OSEBuildingID']).intersection(set(data2016['OSEBuildingID']))
len(buildingID)

La variable OSEBuildingID est bien une clé candidate pour chaque fichier. Il y a 3284 clé communes aux deux fichiers et donc 3284 lignes dont on peut observer une évolution.

### Vérification du contenu des colonnes

In [ ]:
# Jointure des deux dataframes
data = data2016.merge(data2015,how='inner', on='OSEBuildingID', suffixes=(None, '2015'))

### Retrait des colonnes redondantes ou inutiles

In [ ]:
data = data.drop(['DataYear', 'City', 'State', 'TaxParcelIdentificationNumber', 'YearsENERGYSTARCertified', 'ElectricitykWh', 'NaturalGastherms', 'DataYear2015', 'BuildingType2015', 'TaxParcelIdentificationNumber2015', 'CouncilDistrictCode2015', 'PropertyName2015', 'CouncilDistrictCode2015', 'Neighborhood2015', 'YearBuilt2015', 'YearsENERGYSTARCertified2015', 'ElectricitykWh2015', 'NaturalGastherms', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats', 'ZipCode2015'], axis=1)

### Retrait des bâtiments résidentiels

In [ ]:
# Affichage des différents types de bâtiments
plt.figure(figsize=(12,12))
sns.countplot(x=data['BuildingType'])

In [ ]:
data['BuildingType'].unique()

In [ ]:
# Récupération des bâtiments non résidentiels et reset de l'index
data = data.query('BuildingType =="NonResidential" or BuildingType =="Nonresidential COS" or BuildingType =="SPS-District K-12" or BuildingType =="Campus"')
data.reset_index(drop=True, inplace=True)

### Valeurs manquantes

In [ ]:
# Affichage des valeurs manquantes par colonnes en nombre et en ratio
#Initialisation
missingValueNumber = 0
missingValueNumberTotal = 0
missingValuesList = list()

#Boucle de calcul et d'affichage par colonnes
for col in data :
    missingValueNumber = data[col].isna().sum()
    missingValueRatio = missingValueNumber / data[col].size
    print (f'Colonne : {col} : {missingValueNumber} valeurs manquantes, ratio : {round(missingValueRatio,2)}')
    missingValueNumberTotal = missingValueNumberTotal + missingValueNumber
    missingValuesList.append(missingValueRatio)

# Affichage des valeurs manquantes au total en nombre et en ratio
missingValueRatioTotal = missingValueNumberTotal / data.size
print(f'Le data frame comprends {missingValueNumberTotal} valeurs manquantes au total soit un ratio de {round(missingValueRatioTotal,2)}')

Le ratio de valeurs manquantes pour les usages secondaires et tertiaires est égal au ratio de valeurs manquantes de la surface allouée correspondante ce qui suggère qu'il n'y a pas d'erreur de saisie mais des bâtiment n'ayant simplement pas d'usages secondaires et/ou tertiaires. Remplacer ces usages vides par None et la surface par 0. Attention, il y a quelques différences avec 2016, on conserve donc les données des deux relevés au cas où un changement d'activité expliquerait certains changements dans la consommation.

Comments : vide à supprimer

Comments 2015 : presque vide mais peut expliquer d'éventuels outliers.

In [ ]:
# Remplacement des Nan dans les colonnes liéees au 2nd et 3e plus large type d'usages
data[['SecondLargestPropertyUseType','ThirdLargestPropertyUseType','SecondLargestPropertyUseType2015','ThirdLargestPropertyUseType2015']]=data[['SecondLargestPropertyUseType','ThirdLargestPropertyUseType','SecondLargestPropertyUseType2015','ThirdLargestPropertyUseType2015']].fillna(value='none')
data[['SecondLargestPropertyUseTypeGFA','ThirdLargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA2015','ThirdLargestPropertyUseTypeGFA2015']]=data[['SecondLargestPropertyUseTypeGFA','ThirdLargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA2015','ThirdLargestPropertyUseTypeGFA2015']].fillna(value=0)

In [ ]:
#Retrait de la colonne Comments
data = data.drop('Comments', axis=1)

In [ ]:
#Recherche des valeurs manquantes par lignes
#Initialisation
missingValuesList = list()
missingValuesList = data.isna().sum(axis=1)/len(data.columns)

# Création d'un graphique d'estimation de densité par noyau
sns.kdeplot(missingValuesList)
plt.title('Taux de valeurs manquantes')


In [ ]:
# Retrait des lignes ayant plus de 10% de valeurs manquantes
# Création d'un masque
mask = list()
for item in missingValuesList :
    if item >0.10 :
      mask.append(True)
    else :
      mask.append(False)

# Création d'une liste des index à retirer
maskFrame = pd.DataFrame(mask,columns=['toBeRemoved'])
maskFrame = maskFrame.loc[~(maskFrame==False).any(axis=1)]
rowToDrop = maskFrame.index.to_list()
#Retrait des lignes ayant plus de 10% de valeurs manquantes
data = data.drop(rowToDrop, axis=0)
len(rowToDrop)

### Valeurs manquantes : détails

In [ ]:
#Recherche des observations n'ayant pas de valeur de consommation surfacique
data.query('SiteEUIkBtusf != SiteEUIkBtusf')


In [ ]:
data.query('SiteEUIWNkBtusf != SiteEUIWNkBtusf')

In [ ]:
data.query('SiteEnergyUseWNkBtu != SiteEnergyUseWNkBtu')


In [ ]:
# Bâtiments à retirer car manquant de données ou incohérents (ID 757)
indexToRemove = [295, 549]

In [ ]:
# Le minimum de bâtiment par lot est de 1
data['NumberofBuildings']=data['NumberofBuildings'].fillna(value=1)

# Bâtiments à retirer de l'étude car ayant eu des variations d'usage, de surface, travaux en 2015,2016 ou entre ces deux années

Pour les repérer, on va observer les changements majeurs dans la surface, l'usage principal ou dans les commentaires.

In [ ]:
# Affichage des commentaires non vides
data.query('Comments2015 == Comments2015')['Comments2015']

In [ ]:
data['Comments2015'].unique()

In [ ]:
indexToRemove.extend([936])

In [ ]:
# Recherche des propriétés ayant eu une augmentation ou une diminution de surface de plus de 15%
data.query('PropertyGFATotal > PropertyGFATotal2015*1.15 or PropertyGFATotal < PropertyGFATotal2015/1.15')[['PropertyGFATotal', 'PropertyGFATotal2015']]

In [ ]:
# Ajout des propriété ayant vu leur surface évoluer de façon majeure à la liste des lots à retirer de l'étude
indexToRemove.extend(data.query('PropertyGFATotal > PropertyGFATotal2015*1.15 or PropertyGFATotal < PropertyGFATotal2015/1.15')[['PropertyGFATotal', 'PropertyGFATotal2015']].index.to_list())

In [ ]:
# Recherche des propriétés ayant eu variation de l'usage principal en enlevant ceux dont l'usage était indéterminé en 2015
data.query('LargestPropertyUseType != LargestPropertyUseType2015 and LargestPropertyUseType2015 == LargestPropertyUseType2015')[['LargestPropertyUseType', 'LargestPropertyUseType2015']]

In [ ]:
# Recherche de propriétés mal indiquées comme Non-Résidentiel
data.query('PrimaryPropertyType == "Low-Rise Multifamily"')

In [ ]:
#Ajout des index à retirer, dont le changement d'usage principal ne fait pas de doute
indexToRemove.extend([191,237,580,605,1385,1489,1569,628,906,1357])
#Retrait des duplicatas
indexToRemove=list(set(indexToRemove))
# Retrait des lignes
data=data.drop(index=indexToRemove)
# Reinitialisation des index
data.reset_index(drop=True, inplace=True)

# Evaluation de la pertinence des colonnes dans la résolution du problème

En énergétique du bâtiment, on trouve souvent plus pertinent d'étudier la consommation surfacique d'un bâtiment plutôt que sa consommation globale, on étudiera donc plus précisémment les variables par pied carré.(sf)

Les variables qui influencent cette consommation sont :
- les types d'énergie utilisés (rendements variables)
- la compacité des bâtiments (grossièrement estimable avec la surface brute, le nombre d'étages et le nombre de bâtiments)
- les ratios de surface affectés à chaque usages (3 principaux usages + parking)
- l'année de construction ou de rénovation majeure
- la localisation


Variables à créer :
- un ratio d'utilisation pour chaque type d'énergie (en principe, ces ratios peuvent être facilement estimés à partir de l'analyse des installation énergétiques d'un bâtiment, cela dit, le calcul de ce ratio peut aussi être la source de data leakage puisqu'il se base sur des consommations, il faudrait voir la capacité des partenaires à fournir des ratios consistants)
- un indice de compacité du bâtiment : les bâtiments les plus massifs on un rapport surface déperditives/surface au sol plus avantageux : Surface brute totale/Nbre de bâtiment
- une variable par usage avec le ratio de surface brute attribué, c'est une façon d'encoder ces variables avec une variante de "One Hot Encoder" mais qui dépend du ratio de surface utilisé (ces variables feront éventuellement l'objet d'une réduction de dimension)
- pas de modification de la variable existante pour l'année

Concernant le climat, l'ensemble des bâtiments se situent dans la même ville, il n'y a donc que des variation météorologiques négligeables. Cependant, la topographie côtière, humide ainsi que son densité variable peuvent créer des conditions locales sensiblement différentes, il peut donc être intéressant de conserver de conserver la variable des quartiers.

In [ ]:
#Suppression des colonnes inutiles
data = data.drop(['Address', 'ZipCode', 'CouncilDistrictCode', 'Latitude', 'Longitude', 'DefaultData', 'ComplianceStatus', 'PrimaryPropertyType2015', 'Location', 'DefaultData2015', 'Comments2015', 'ComplianceStatus2015'], axis=1)

### Formatage

In [ ]:
#Définition de fonction de formatage
def lower(stringToLower):
  #Renvoie la chaine de caractères en minuscule
  return stringToLower.lower()

#Application de la fonction de formatage
data['Neighborhood'] = data['Neighborhood'].apply(lower)

## Feature engineering

In [ ]:
# Création de la variable 'Compacité'
data['Compacity'] = [(row[11]/max(row[7],1)) for row in data.itertuples()]

In [ ]:
# Création des variables "ratios d'utilisation d'un type d'énergie"
data['SteamRatio'] = [min(1,(row[26]/max(row[24],1))) for row in data.itertuples()]
data['ElectricityRatio'] = [min(1,(row[27]/max(1,row[24]))) for row in data.itertuples()]
data['NaturalGasRatio'] = [min(1,(row[28]/max(1,row[24]))) for row in data.itertuples()]

In [ ]:
# Création des variables "ratios d'usages des surfaces'
# Récupération de tous les usages possibles
useTypes = data['LargestPropertyUseType']
useTypes.append(data['SecondLargestPropertyUseType'])
useTypes.append(data['ThirdLargestPropertyUseType'])
useTypes = useTypes.unique()

# Remplissage des ratios à partir des 3 usages les plus importants
for types in useTypes :
    ratioType = list()
    for row in data.itertuples():
        if row[13]==types:
            ratioType.append(min(1,row[14]/max(1,row[9])))
        elif row[15]==types:
            ratioType.append(min(1,row[16]/max(1,row[9])))
        elif row[17]==types:
            ratioType.append(min(1,row[18]/max(1,row[9])))
        else:
            ratioType.append(0)
    data[types]=ratioType
    
# Colonne spécifique aux parkings
ratioType = list()
for row in data.itertuples():
    ratioType.append(min(1, row[10]/max(1,row[9])))
data['Parking']=ratioType

In [ ]:
# Graphiques boxplot de certains indicateurs
plt.figure(figsize=(20,14))
sns.set({"font.size":16,"axes.titlesize":16,"axes.labelsize":16}, style='white')

# Initialisation de la position du graphique
i=1
# Variables à afficher
featuresToBoxplot = ['YearBuilt', 'PropertyGFATotal','ENERGYSTARScore','SiteEUIkBtusf','SiteEnergyUsekBtu', 'TotalGHGEmissions']

# Boucle permettant de créer un graphique par indicateur
for feature in featuresToBoxplot :
    # Emplacement itératif du graphique
    plt.subplot(6,1,i)
    sns.boxplot(x=data[feature], orient='h')
    plt.title(feature)
    plt.tight_layout()
    # Incrémentation de la position
    i+=1


In [ ]:
#Matrice des corrélations
corrDf = data[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal','PropertyGFAParking','PropertyGFABuildings','ENERGYSTARScore','SiteEUIkBtusf','SiteEUIWNkBtusf','SourceEUIkBtusf','SourceEUIWNkBtusf','SiteEnergyUsekBtu','SiteEnergyUseWNkBtu','SteamUsekBtu','ElectricitykBtu','NaturalGaskBtu','TotalGHGEmissions','GHGEmissionsIntensity','SiteEnergyUsekBtu2015','TotalGHGEmissions2015']].corr(method='pearson')

plt.figure(figsize=(20, 20))
sns.heatmap(corrDf, annot=True)
plt.show()
# Sauvegarde des graphiques
plt.savefig('Corr1.png')

Corrélations logiques dûes à des relations mathématiques entre les variables : 
 - Energie Site/Source (Finale/Primaire en français) correspond à un équivalent d'énergie fossile de l'énergie réellement utilisée. 

 - Energie WN correspond à une normalisation de la consommation en fonction d'un climat de référence.
 
 - GHGEmissions : Ces valeurs sont généralement calculées à partir des usages particuliers d'énergie (steam, natural gas, elecricity) 


Variables corrélées avec la consommation énergétique totale (SiteEnergyUsekBtu): Dimensions du lot (surface, étage, nombre de bâtiments), année... On constate logiquement que la consommation a une corrélation plus forte avec la surface bâtie qu'avec la surface de parking ce qui est logique, la consommation surfacique des parkings étant bien moindre.

EnergyStarScore : Très peu corrélé à la consommation totale, il l'est logiquement plus avec la consommation par unité de surface. Il est corrêlé négativement puisque un score proche de 100 corresponds à un bâtiment plus économique.

In [ ]:
data.head(10)

In [ ]:
#Matrice des corrélations réduite : uniquement les valeurs surfaciques et retrait des variables mathématiquement corrêlées et ajout des variables créées
corrDf = data[['SiteEUIkBtusf','YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal','PropertyGFAParking','Compacity','PropertyGFABuildings','ENERGYSTARScore','SteamRatio','ElectricityRatio','NaturalGasRatio','SteamUsekBtu','ElectricitykBtu','NaturalGaskBtu' ]].corr(method='pearson')

plt.figure(figsize=(20, 20))
sns.heatmap(corrDf, annot=True)
plt.show()
# Sauvegarde des graphiques
plt.savefig('Corr2.png')

Les corrélations entre les variables créées par feature engineering et la consommation d'énergie par unité de surface n'est pas très élevée mais cela ne permet pas de les disqualifier puisque leur influence peut-être moins linéaire ou dépendante d'autres variables.

# Représentations bi-variées (notamment des variables créées)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Compacity'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Compacity'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('compacity.png')

In [ ]:
# Graphique
graph = sns.jointplot(x=data['SteamRatio'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['SteamRatio'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('steamratio.png')

In [ ]:
# Graphique
graph = sns.jointplot(x=data['NaturalGasRatio'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['NaturalGasRatio'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('gasratio.png')

In [ ]:
# Graphique
graph = sns.jointplot(x=data['ElectricityRatio'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['ElectricityRatio'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('electricityratio.png')

Concernant les ratios d'usage, il est logique d'observer une baisse de la consommation d'énergie générale lorsque l'électricité est prépondérante, en effet l'électricité est une énergie noble, elle peut être utilisée pour de nombreux usages avec très peu de pertes, son usage permet donc de réduire la quantité d'énergie finale requise. (Notons en revanche qu'elle requiert beaucoup d'énergie primaire pour être produite, cette "noblesse" a un coût qui se situe plus au niveau de la production et de l'acheminement)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['YearBuilt'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['YearBuilt'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('yearbuilt.png')

Logiquement, des bâtiments plus récents devraient présenter une consommation inférieure vu l'amélioration de l'efficacité des isolants, des éclairages etc... Or on constate l'inverse, cela peut-être dû au fait que les bâtiments ont évolué en usage au cours des années, que les bâtiments plus modernes sont plus équipés ou que les bâtiments moins utilisés sont aussi moins souvent rénovés.

In [ ]:
# Graphique
graph = sns.jointplot(x=data['ENERGYSTARScore'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['ENERGYSTARScore'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)
# Sauvegarde des graphiques
plt.savefig('Estar.png')

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Hotel'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Hotel'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Data Center'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Data Center'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Office'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Office'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Parking'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Parking'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)

In [ ]:
# Graphique
graph = sns.jointplot(x=data['Retail Store'], y=data['SiteEUIkBtusf'], height=10)

# Regression linéaire
sns.regplot(x=data['Retail Store'], y=data['SiteEUIkBtusf'], scatter=False, ax=graph.ax_joint)

Les tendances montrées liées au ratio d'usages montrent logiquement que les data centers présentent des consommations énergétiques élevées. Concernant les parkings, l'augmentation contre intuitive de consommation liée à l'augmentation du ratio a révélé (et les calculs le confirment) que la valeur indiquée de SiteEUIkBtusf est calculée sur la surface bâtie hors parkings, dans le cas contraire, la présence de parkings aurait dû faire baisser la consommation surfacique puisque les parkings consomment nettement moins d'énergie que les autres usages.

# Outliers

Certaines observations affichent des valeurs très élevées qui bien que réalistes sont atypiques, pour éviter que ces outliers n'aient une trop grande influence sur les algorithmes de machine learning et notamment des regressions, nous allons supprimmer ces observation, les bâtiments de ce type devront faire l'objet d'une étude particulière.

In [ ]:
data.head()

In [ ]:
# Suppression des observations présentant des valeurs de consommation énergétique surfacique et une intensité d'émission GHG supérieure ou inférieure de 3 fois l'écart type par rapport à la moyenne
# Bien que n'étant pas irréalistes, ces valeurs peuvent trop influencer le modèle et induire plus d'erreurs sur la grande majorité des lots étudiés

#Initialisation
data.reset_index(drop=True, inplace=True)

# Retrait des lignes
# Création d'un masque
mask = list()
for row in data.itertuples() :
    if row[20] > data['SiteEUIkBtusf'].mean() + 3 * data['SiteEUIkBtusf'].std() or row[20] < data['SiteEUIkBtusf'].mean() - 3 * data['SiteEUIkBtusf'].std():
        mask.append(True)
    elif row[31] > data['GHGEmissionsIntensity'].mean() + 3 * data['GHGEmissionsIntensity'].std() or row[31] < data['GHGEmissionsIntensity'].mean() - 3 * data['GHGEmissionsIntensity'].std():
        mask.append(True)
    else :
        mask.append(False)
    

# Création d'une liste des index à retirer
maskFrame = pd.DataFrame(mask,columns=['toBeRemoved'])
maskFrame = maskFrame.loc[~(maskFrame==False).any(axis=1)]
rowToDrop = maskFrame.index.to_list()
#Retrait des lignes ayant plus de 10% de valeurs manquantes
data = data.drop(rowToDrop, axis=0)
len(rowToDrop)


# Remplacement des Nan :

In [ ]:
# Affichage des valeurs manquantes par colonnes en nombre et en ratio
#Initialisation
missingValueNumber = 0
missingValueNumberTotal = 0
missingValuesList = list()

#Boucle de calcul et d'affichage par colonnes
for col in data :
    missingValueNumber = data[col].isna().sum()
    missingValueRatio = missingValueNumber / data[col].size
    print (f'Colonne : {col} : {missingValueNumber} valeurs manquantes, ratio : {round(missingValueRatio,2)}')
    missingValueNumberTotal = missingValueNumberTotal + missingValueNumber
    missingValuesList.append(missingValueRatio)

# Affichage des valeurs manquantes au total en nombre et en ratio
missingValueRatioTotal = missingValueNumberTotal / data.size
print(f'Le data frame comprends {missingValueNumberTotal} valeurs manquantes au total soit un ratio de {round(missingValueRatioTotal,2)}')

A ce stade, un examen des valeurs manquantes révèle qu'il n'y en a aucune à traiter. En particulier, on ne souhaite pas remplir les valeurs manquantes pour L'ENERGYSTARScore puisqu'on veut l'évaluer par la suite.

# Encodage, normalisation et lissage

In [ ]:
data.head(1)

Variables à encoder ordinalement : Aucune



Variables à encoder par la méthode "One Hot Encoder" : Neighborhood

Variable à mettre à l'échelle : YearBuilt, NumberOfFloors, NumberOfBuildings, Variables figurant des surfaces


Variables à lisser (logarithmique) : SiteEUIkBtusf GHGEmissionsIntensity

Remarque : toutes les variables correspondant aux consommations ou aux GHG émis découlent presque directement de la consommation totale, si l'objet de l'algorithme de prédiction est de prédire la consommation totale alors il est logique que ces autres variables ne peuvent être prises en compte pour la prédiction, l'algorithme de ML doit utiliser des informations plus facilement accessible. Autrement dit : vu le mode de calcul des émissions GHG et des consos Source ou sur un type d'énergie particulière, l'absence d'info sur la consommation totale induit forcément une absence d'info sur ces consommations. On conservera cependant les ratios d'usage d'énergie pour prédire car s'il sont ici calculés à partir de ces consommations, ils peuvent aussi être estimés au vu des installations du lot.

In [ ]:
# One Hot Encoder
data = pd.get_dummies(data, columns=['Neighborhood'])

In [ ]:
# Mise à l'échelle
from sklearn.preprocessing import MinMaxScaler
myScaler = MinMaxScaler()
data[['YearBuilt', 'NumberofFloors', 'NumberofBuildings','PropertyGFATotal','PropertyGFAParking','PropertyGFABuildings','ENERGYSTARScore','Compacity']] = myScaler.fit_transform(data[['YearBuilt', 'NumberofFloors', 'NumberofBuildings','PropertyGFATotal','PropertyGFAParking','PropertyGFABuildings','ENERGYSTARScore','Compacity']])

In [ ]:
# Lissage logarithmique des valeurs à prédire
data[['SiteEUIkBtusf.log10','GHGEmissionsIntensity.log10']] = np.log10(data[['SiteEUIkBtusf','GHGEmissionsIntensity']]+1)

# Dernier nettoyage des variables et enregistrement du jeu de données

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
#Constitution de la liste des variables non traitées dans le modèle
columnsToRemove = data.columns.to_list()
del columnsToRemove[4:10]
del columnsToRemove[11:12]
del columnsToRemove[50:]
columnsToRemove

In [ ]:
#Retrait de ces variables
data = data.drop(columnsToRemove, axis=1)

In [ ]:
#Réorganisation de l'ordre des colonnes
cols = list(data.columns.values)
cols.pop(cols.index('ENERGYSTARScore'))
cols.pop(cols.index('SiteEUIkBtusf.log10'))
cols.pop(cols.index('GHGEmissionsIntensity.log10'))
data = data[cols+['ENERGYSTARScore','SiteEUIkBtusf.log10','GHGEmissionsIntensity.log10']]
data.head()

Voilà qui conclut la préparation de notre jeu de données

In [ ]:
# Export du dataFrame de travail en fichier csv
data.to_csv('dataCleanP4.csv', index = False)